# B4

### (1) Add parts-of-speech bigrams to the bag-of-words

In [102]:
pos_bigram = list(nltk.bigrams(pos))
pattern = [('JJ', 'NN'),('JJ','NNS'),('NN','NNS')]
import copy
corpus_words_bi = copy.deepcopy(corpus_words)

for i in range(len(pos_bigram)):
    (a,b),(c,d) = pos_bigram[i]
    if (b,d) in pattern:
        corpus_words_bi.append(' '.join(corpus_words[i:i+len(pattern)]))

### (2) Define the function to create new features

In [ ]:
def document_features_bigram(document):
    document_words = set(document)
    features = {}
    for wd in corpus_words_bi:
        features['contains({})'.format(wd)] = (wd in document_words)
    return features

### (3) Find out the unigrams and bigrams in each job description that match the part-of-speech patterns we defined

In [103]:


for i in range(len(fullset)):
    words = re.sub(r'[^\w\s]','',fullset.FullDescription[i])
    tokens = nltk.word_tokenize(words.lower().decode('utf-8'))
    tokens = [w for w in tokens if w.isalpha()==True]
    tags = nltk.pos_tag(tokens)
    bigram = list(nltk.bigrams(tags))
    for n in range(len(bigram)):
        (a,b),(c,d) = bigram[n]
        if (b,d) in pattern:
            tokens.append(' '.join(tokens[n:n+len(pattern)]))
    feature = document_features_bigram(tokens)
    fullset.token[i] = feature

### (4) Run classification model

In [104]:
fullset['feature_sets'] = zip(fullset.token, fullset.category)
featuresets = fullset.feature_sets.tolist()
train_set, test_set = featuresets[:500], featuresets[500:]
classifier = nltk.NaiveBayesClassifier.train(train_set)
nltk.classify.accuracy(classifier, test_set)

0.808

# My version of B2

### (1) Lemmatize words according to their pos in the corpus

In [121]:
from nltk.stem import WordNetLemmatizer
wnl = WordNetLemmatizer()

for i in range(len(fullset)):
    words = re.sub(r'[^\w\s]','',fullset.FullDescription[i])
    token = nltk.word_tokenize(words.lower().decode('utf-8'))
    pos_i = nltk.pos_tag(token)
    wordnet_tag ={'NN':'n','JJ':'a','VB':'v','RB':'r'}
    resultList = []
    for t in pos_i:
        try: resultList.append(wnl.lemmatize(t[0],wordnet_tag[t[1][:2]]))
        except: resultList.append(wnl.lemmatize(t[0]))
    feature = document_features(resultList)
    fullset.token[i] = feature

### (2) Run classification model

In [122]:
fullset['feature_sets'] = zip(fullset.token, fullset.category)
featuresets = fullset.feature_sets.tolist()
train_set, test_set = featuresets[:500], featuresets[500:]
classifier = nltk.NaiveBayesClassifier.train(train_set)

In [123]:
nltk.classify.accuracy(classifier, test_set)

0.818